In [1]:
%load_ext autoreload
%autoreload 2

from spalah.dataframe import slice_dataframe, SchemaComparer
from pyspark.sql import SparkSession, DataFrame

In [2]:
spark = SparkSession.builder.getOrCreate()

22/07/17 18:40:11 WARN Utils: Your hostname, Alexandrs-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.68.121 instead (on interface en0)
22/07/17 18:40:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/07/17 18:40:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
from datetime import datetime, date

from pyspark.sql import Row

df = spark.createDataFrame([
    Row(a=1, b=2., c='string1', d=date(2000, 1, 1), e=datetime(2000, 1, 1, 12, 0)),
    Row(a=2, b=3., c='string2', d=date(2000, 2, 1), e=datetime(2000, 1, 2, 12, 0)),    
])
df.show()

In [ ]:
from spalah.dataframe import slice_dataframe

slice_dataframe(
    input_dataframe=df,
    columns_to_include=["a", "b", "c", "d"],    
    columns_to_exclude=[],
    nullify_only=False
).printSchema()

In [ ]:
nested_df = spark.sql("""
select 
    1 as column_a
,   2.0 as column_b
,   struct(
        "c1" as column_c_1
    ,   struct(
            "c_2_1" as c_2_1,
            "c_2_2" as c_2_2,
            "c_2_3" as c_2_3
    ) as column_c_2
) as column_c
""")

In [ ]:
nested_df.printSchema()

In [ ]:
slice_dataframe(
    input_dataframe=nested_df,
    columns_to_include=["column_c"],
    columns_to_exclude=["column_c.column_c_1"],
    nullify_only=False,
    debug=False
).select("column_c.*").show(10, False)

### SchemaComparer

In [3]:
df_source = spark.sql(
    'SELECT 1 as ID, "John" AS Name, struct("line1" AS Line1, "line2" AS Line2) AS Address'
)
df_source.printSchema()

df_target = spark.sql(
    'SELECT "a" as ID, "John" AS name, struct("line1" AS Line1) AS Address'
)
df_target.printSchema()




root
 |-- ID: integer (nullable = false)
 |-- Name: string (nullable = false)
 |-- Address: struct (nullable = false)
 |    |-- Line1: string (nullable = false)
 |    |-- Line2: string (nullable = false)

root
 |-- ID: string (nullable = false)
 |-- name: string (nullable = false)
 |-- Address: struct (nullable = false)
 |    |-- Line1: string (nullable = false)



In [4]:
schema_comparer = SchemaComparer(
    source_schema = df_source.schema,
    target_schema = df_target.schema
)

schema_comparer.compare()

schema_comparer.matched
schema_comparer.not_matched